In [1]:
import pandas as pd
import gspread
import os
import requests

from oauth2client.service_account import ServiceAccountCredentials

In [2]:
def connect_to_twitter():
    bearer_token = os.environ.get("BEARER_TOKEN")
    return {"Authorization": "Bearer {}".format(bearer_token)}

In [3]:
headers = connect_to_twitter()

In [8]:
def make_request(headers):
    url = "https://api.twitter.com/2/users/2244994945/tweets"
    params = {
        "tweet.fields": "created_at,lang",
        "max_results": "100"
    }
    return requests.request("GET", url, headers=headers, params=params).json()

In [9]:
response = make_request(headers)
print(response)

{'data': [{'created_at': '2021-03-22T21:04:00.000Z', 'lang': 'en', 'id': '1374104599456534531', 'text': 'Hope to see you tomorrow at 1 pm EST for APIs 101! \nhttps://t.co/GrtBOXyHmB https://t.co/YyQfmgiLlL'}, {'created_at': '2021-03-19T19:59:10.000Z', 'lang': 'en', 'id': '1373001119480344583', 'text': 'Looking to get started with the Twitter API but new to APIs in general? @jessicagarson will walk you through everything you need to know in APIs 101 session. She’ll use examples using our v2 endpoints, Tuesday, March 23rd at 1 pm EST.\n\nJoin us on Twitch\nhttps://t.co/GrtBOXyHmB'}, {'created_at': '2021-03-18T19:15:37.000Z', 'lang': 'en', 'id': '1372627771717869568', 'text': "Thanks to everyone who joined and made today a great session! 🙌 \n\nIf weren't able to attend, we've got you covered. Academic researchers can now sign up for office hours for help using the new product track. See how you can sign up, here 👇\nhttps://t.co/duIkd27lPx https://t.co/AP9YY4F8FG"}, {'created_at': '2021-03

In [10]:
def make_df(response):
    return pd.DataFrame(response["data"])

In [11]:
df = make_df(response)
df

,created_at,lang,id,text
0,2021-03-22T21:04:00.000Z,en,1374104599456534531,Hope to see you tomorrow at 1 pm EST for APIs ...
1,2021-03-19T19:59:10.000Z,en,1373001119480344583,Looking to get started with the Twitter API bu...
2,2021-03-18T19:15:37.000Z,en,1372627771717869568,Thanks to everyone who joined and made today a...
3,2021-03-18T19:07:02.000Z,en,1372625612460810242,"@geet_qcp @suhemparack Hey! We have good news,..."
4,2021-03-18T16:50:09.000Z,en,1372591165006999556,We're going to get started in just a few minut...
...,...,...,...,...
95,2020-09-08T18:54:48.000Z,en,1303406450757251072,RT @jessicagarson: Published another blog post...
96,2020-09-03T17:31:39.000Z,en,1301573587187331074,"Starting today, you can see your monthly Tweet..."
97,2020-09-03T15:43:00.000Z,en,1301546240887398401,"RT @snowman: So, we built a live golf leaderbo..."
98,2020-09-01T20:07:50.000Z,en,1300888112948752389,⛳ Why do golfers carry an extra shirt? In case...


In [12]:
def authenticate_to_google():
    scope = [
        "https://spreadsheets.google.com/feeds"
    ]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        "Downloads/cred_new.json", scope
    )
    return credentials

In [13]:
credentials = authenticate_to_google()
gc = gspread.authorize(credentials)
workbook = gc.open_by_key("1xxuZ3hhcqCOqYmJaMckZ3mvBOkBzjNZYjBiMpIzg9Ug")
sheet = workbook.worksheet("Testing")
sheet.update("A1", [df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1xxuZ3hhcqCOqYmJaMckZ3mvBOkBzjNZYjBiMpIzg9Ug',
 'updatedRange': 'Testing!A1:D101',
 'updatedRows': 101,
 'updatedColumns': 4,
 'updatedCells': 404}